In [1]:
# Install the model and dependencies
# !git clone https://huggingface.co/xingyang1/Distill-Any-Depth
# %cd Distill-Any-Depth
# !pip install -r requirements.txt
# %cd ../
!git clone https://huggingface.co/spaces/xingyang1/Distill-Any-Depth

Cloning into 'Distill-Any-Depth'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 137 (delta 33), reused 0 (delta 0), pack-reused 4 (from 1)
Receiving objects: 100% (137/137), 111.55 KiB | 2.14 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Filtering content: 100% (10/10), 23.52 MiB | 9.08 MiB/s, done.


In [1]:
%cd Distill-Any-Depth
import torch
from torchvision import datasets, transforms
from PIL import Image
import numpy as np
import os
import cv2

from distillanydepth.modeling.archs.dam.dam import DepthAnything
from distillanydepth.utils.image_util import colorize_depth_maps
from distillanydepth.midas.transforms import Resize, NormalizeImage, PrepareForNet
from torchvision.transforms import Compose
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

/home/kothamachuharish.g/.conda/envs/pytorch_env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/kothamachuharish.g/Distill-Any-Depth


/home/kothamachuharish.g/.conda/envs/pytorch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
xFormers not available
xFormers not available
xFormers not available
xFormers not available


In [5]:
# Paths
os.makedirs("cifar_images", exist_ok=True)
os.makedirs("cifar_depths", exist_ok=True)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CIFAR-10 images
transform_cifar = transforms.Compose([
    transforms.ToTensor()
])
cifar_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_cifar)

In [6]:
# Get 100 images
cifar_images = [transforms.ToPILImage()(img) for img, _ in list(cifar_dataset)[:9999]]

# Prepare depth model
model_kwargs = dict(
    encoder="vitl",
    features=256,
    out_channels=[256, 512, 1024, 1024],
    use_bn=False,
    use_clstoken=False,
    max_depth=150.0,
    mode='disparity',
    pretrain_type='dinov2',
    del_mask_token=False
)

model = DepthAnything(**model_kwargs).to(device)

# Load model weights
checkpoint_path = hf_hub_download(
    repo_id="xingyang1/Distill-Any-Depth",
    filename="large/model.safetensors",
    repo_type="model"
)
model_weights = load_file(checkpoint_path)
model.load_state_dict(model_weights)
# model.eval() # shows the entire model architecture

<All keys matched successfully>

In [4]:
# Preprocessing pipeline
preprocess = Compose([
    Resize(756, 756, resize_target=False, keep_aspect_ratio=True, ensure_multiple_of=14,
           resize_method='lower_bound', image_interpolation_method=cv2.INTER_CUBIC),
    NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    PrepareForNet()
])

In [7]:
# Generate depth maps
for idx, img in enumerate(cifar_images):
    # Save original image
    img_path = f"cifar_images/img_{idx:04d}.png"
    img.save(img_path)

    # Preprocess image
    img_np = np.array(img)[..., ::-1] / 255.0  # BGR
    sample = preprocess({'image': img_np})
    image_tensor = torch.from_numpy(sample['image']).unsqueeze(0).to(device)

    # Predict depth
    with torch.no_grad():
        pred_disp, _ = model(image_tensor)

    # Normalize and colorize depth map
    depth_np = pred_disp[0, 0].cpu().numpy()
    depth_norm = (depth_np - depth_np.min()) / (depth_np.max() - depth_np.min())
    depth_color = colorize_depth_maps(depth_norm[None, ..., None], 0, 1, cmap="Spectral_r").squeeze()
    depth_color = (depth_color * 255).astype(np.uint8)
    depth_color_hwc = depth_color.transpose(1, 2, 0)  # CHW to HWC

    # Resize back to original CIFAR-10 size
    depth_color_hwc = cv2.resize(depth_color_hwc, (32, 32), interpolation=cv2.INTER_LINEAR)

    # Save depth map
    depth_path = f"cifar_depths/depth_{idx:04d}.png"
    cv2.imwrite(depth_path, cv2.cvtColor(depth_color_hwc, cv2.COLOR_RGB2BGR))

    print(f"[{idx+1}/100] Saved: {depth_path}")

[1/100] Saved: cifar_depths/depth_0000.png
[2/100] Saved: cifar_depths/depth_0001.png
[3/100] Saved: cifar_depths/depth_0002.png
[4/100] Saved: cifar_depths/depth_0003.png
[5/100] Saved: cifar_depths/depth_0004.png
[6/100] Saved: cifar_depths/depth_0005.png
[7/100] Saved: cifar_depths/depth_0006.png
[8/100] Saved: cifar_depths/depth_0007.png
[9/100] Saved: cifar_depths/depth_0008.png
[10/100] Saved: cifar_depths/depth_0009.png
[11/100] Saved: cifar_depths/depth_0010.png
[12/100] Saved: cifar_depths/depth_0011.png
[13/100] Saved: cifar_depths/depth_0012.png
[14/100] Saved: cifar_depths/depth_0013.png
[15/100] Saved: cifar_depths/depth_0014.png
[16/100] Saved: cifar_depths/depth_0015.png
[17/100] Saved: cifar_depths/depth_0016.png
[18/100] Saved: cifar_depths/depth_0017.png
[19/100] Saved: cifar_depths/depth_0018.png
[20/100] Saved: cifar_depths/depth_0019.png
[21/100] Saved: cifar_depths/depth_0020.png
[22/100] Saved: cifar_depths/depth_0021.png
[23/100] Saved: cifar_depths/depth_0022.p